<a href="https://colab.research.google.com/github/nordluma/yoloCustomObjectDetection/blob/main/inl%C3%A4mning8_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
ln: failed to create symbolic link '/mydrive/My Drive': File exists
 appdata	     Assignment2.ipynb	'My Drive'		      yolov3
 assignment2.ipynb  'Colab Notebooks'	'penkkitreeni 08_2020.xlsx'


## 1. Clone darknet från GitHub

In [ ]:
!git clone https://github.com/AlexeyAB/darknet

fatal: destination path 'darknet' already exists and is not an empty directory.


## 2. Kompilerar darknet med GPU

Sätter upp darknet så att den har tillgång till OpenCV och GPU

In [ ]:
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:926:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1127:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
             ^~
./src/image_opencv.cpp:1130

## 3. Kofigurera darknet för yolo v3

Använder mig av samma inställningar som i tutorialen eftersom de använder också bara en klass

In [ ]:
!cp cfg/yolov3.cfg cfg/yolov3_training.cfg

In [ ]:
!sed -i 's/batch=1/batch=64/' cfg/yolov3_training.cfg
!sed -i 's/subdivisions=1/subdivisions=16/' cfg/yolov3_training.cfg
!sed -i 's/max_batches = 500200/max_batches = 4000/' cfg/yolov3_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_training.cfg

In [ ]:
!mkdir '/mydrive/yolov3'

mkdir: cannot create directory ‘/mydrive/yolov3’: File exists


In [ ]:
!echo 'Turtle' > data/obj.names
!echo -e 'classes = 1\ntrain = data/train.txt\nvalid = data/test.txt\nnames = data/obj.names\nbackup = /mydrive/yolov3' > data/obj.data

In [ ]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2021-03-20 15:26:52--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74.2’

darknet53.conv.74.2 100%[===================>] 154.96M  76.5MB/s    in 2.0s    

2021-03-20 15:26:54 (76.5 MB/s) - ‘darknet53.conv.74.2’ saved [162482580/162482580]



## 4. Extract Images

Unzippar bilderna från google drive

In [ ]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
replace data/obj/animals-zoo-park-african-animals-africa-animal-turtle.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: data/obj/animals-zoo-park-african-animals-africa-animal-turtle.jpg  
  inflating: data/obj/animals-zoo-park-african-animals-africa-animal-turtle.txt  
  inflating: data/obj/animal-turtle-reptile-armored-tortoise-nature-wild-world-marine-amphibian.jpg  
  inflating: data/obj/animal-turtle-reptile-armored-tortoise-nature-wild-world-marine-amphibian.txt  
  inflating: data/obj/apiirkrtk__92482.1612894184.jpg  
  inflating: data/obj/apiirkrtk__92482.1612894184.txt  
  inflating: data/obj/apiodmdrq__41090.1594402782.jpg  
  inflating: data/obj/apiodmdrq__41090.1594402782.txt  
  inflating: data/obj/april_fish_where_to_get_snapping_turtle_in_animal_crossing_new_h_1533988.jpg  
  inflating: data/obj/april_fish_where_to_get_snapping_turtle_in_animal_crossing_new_h_1533988.txt  
  inflating: data/obj/aztec-animal-clinic-exotic-anim

Hämtar alla paths till txt filerna.

In [ ]:
import glob
import os
import re

txt_files_paths = glob.glob(r"data/obj/*.txt")

for i, file_path in enumerate(txt_files_paths):
  with open(file_path, "r") as f_o:
    lines = f_o.readlines()

    text_converted = []
    for line in lines:
      print(line)
      numbers = re.findall("[0-9.]+", line)
      print(numbers)

      if (numbers):
        text = '{} {} {} {} {}'.format(0, numbers[1], numbers[2], numbers[3], numbers[4])
        text_converted.append(text)
        print(i, file_path)
        print(text)
    
    with open(file_path, 'w') as fp:
      for item in text_converted:
        fp.writelines('%s\n' % item)

15 0.445000 0.538690 0.643333 0.565476

['15', '0.445000', '0.538690', '0.643333', '0.565476']
0 data/obj/image15.txt
0 0.445000 0.538690 0.643333 0.565476
15 0.500625 0.820625 0.998750 0.358750

['15', '0.500625', '0.820625', '0.998750', '0.358750']
1 data/obj/turtle20.txt
0 0.500625 0.820625 0.998750 0.358750
15 0.381660 0.474499 0.308402 0.351548

['15', '0.381660', '0.474499', '0.308402', '0.351548']
2 data/obj/turtle1.txt
0 0.381660 0.474499 0.308402 0.351548
15 0.243529 0.170290 0.454118 0.335749

['15', '0.243529', '0.170290', '0.454118', '0.335749']
3 data/obj/turtle12.txt
0 0.243529 0.170290 0.454118 0.335749
15 0.652941 0.222222 0.571765 0.265700

['15', '0.652941', '0.222222', '0.571765', '0.265700']
3 data/obj/turtle12.txt
0 0.652941 0.222222 0.571765 0.265700
15 0.283529 0.508454 0.552941 0.301932

['15', '0.283529', '0.508454', '0.552941', '0.301932']
3 data/obj/turtle12.txt
0 0.283529 0.508454 0.552941 0.301932
15 0.649412 0.510870 0.527059 0.321256

['15', '0.649412', '

In [ ]:
images_list = glob.glob('data/obj/*.jpg')
print(images_list)

['data/obj/turtle-turtle-toy-animals-toy-toys-cute-game-animal.jpg', 'data/obj/box_turtle_web-1024x683.jpg', 'data/obj/images27.jpg', 'data/obj/images80.jpg', 'data/obj/images55.jpg', 'data/obj/images73.jpg', 'data/obj/images32.jpg', 'data/obj/image26.jpg', 'data/obj/cute-happy-turtle-swimming-cartoon-animal-sporty-icon-concept-isolated-flat-cartoon-style_138676-2198.jpg', 'data/obj/turtle-totem-300.jpg', 'data/obj/turtle-spirit-animal1.jpg', 'data/obj/turtle-164153.jpg', 'data/obj/images50.jpg', 'data/obj/MBS_RedTide_PR_090919_jk.jpg', 'data/obj/images25.jpg', 'data/obj/images69.jpg', 'data/obj/green_turtle_high_res_mdv.jpg', 'data/obj/turtle-nature-animal-carapace-little-turtle.jpg', 'data/obj/images24.jpg', 'data/obj/images78.jpg', 'data/obj/images2.jpg', 'data/obj/image16.jpg', 'data/obj/turtle22.jpg', 'data/obj/image20.jpg', 'data/obj/images66.jpg', 'data/obj/turtle11.jpg', 'data/obj/TL4780-sea-turtle-1_1024x1024.jpg', 'data/obj/images38.jpg', 'data/obj/images36.jpg', 'data/obj/Pa

In [ ]:
file = open('data/train.txt', 'w')
file.write('\n'.join(images_list))
file.close()

## 5. Börjar inträningen

In [ ]:
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.922477), count: 4, class_loss = 0.000001, iou_loss = 0.021009, total_loss = 0.021011 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 418056, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.895158), count: 4, class_loss = 0.000002, iou_loss = 0.057732, total_loss = 0.057733 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.0